In [1]:
import gradio as gr
import os
import time

d:\software\anaconda\anaconda3\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def reinit_model(large_language_model, embedding_model, history):
    try:
        knowladge_based_chat_llm.init_model_config(
            large_language_model=large_language_model,
            embedding_model=embedding_model)
        model_status = """模型已成功重新加载，可以开始对话"""
    except Exception as e:

        model_status = """模型未成功重新加载，请点击重新加载模型"""
    return history + [[None, model_status]]

In [3]:
def clear_session():
    return '', None

In [4]:
def init_vector_store(file_obj):

    vector_store = knowladge_based_chat_llm.init_knowledge_vector_store(
        file_obj.name)

    return vector_store

In [5]:
def predict(input,
            use_web,
            top_k,
            history_len,
            temperature,
            top_p,
            history=None):
    if history == None:
        history = []

    resp = knowladge_based_chat_llm.get_knowledge_based_answer(
        query=input,
        web_content=web_content,
        top_k=top_k,
        history_len=history_len,
        temperature=temperature,
        top_p=top_p,
        history=history)
    history.append((input, resp['result']))
    return '', history, history

In [6]:
block = gr.Blocks()
with block as demo:

    model_status = gr.State("欢迎使用CASGPT")#每次重启都会导致State的变量消失
    with gr.Row():
        with gr.Column(scale=1):
            # 这里不需要模型选择。只使用chatglm2-6b
            model_argument = gr.Accordion("模型参数配置")
            with model_argument:
                top_k = gr.Slider(1,
                                    10,
                                    value=6,
                                    step=1,
                                    label="vector search top k",
                                    interactive=True)

                history_len = gr.Slider(0,
                                        5,
                                        value=3,
                                        step=1,
                                        label="history len",
                                        interactive=True)

                temperature = gr.Slider(0,
                                        1,
                                        value=0.01,
                                        step=0.01,
                                        label="temperature",
                                        interactive=True)
                top_p = gr.Slider(0,
                                    1,
                                    value=0.9,
                                    step=0.1,
                                    label="top_p",
                                    interactive=True)

            file = gr.File(label='请上传知识库文件',
                            file_types=['.txt', '.md', '.docx', '.pdf'],
                            height=170)

            init_vs = gr.Button("知识库文件向量化")

        with gr.Column(scale=4):
            chatbot = gr.Chatbot([[None, model_status.value]],
                                    label='ChatLLM')
            message = gr.Textbox(label='请输入问题')
            state = gr.State()

            with gr.Row():
                clear_history = gr.Button("🧹 清除历史对话")
                send = gr.Button("🚀 发送")

        init_vs.click(#点击一下init_vs就会触发init_vector_store函数
            init_vector_store,
            show_progress=True,
            inputs=[file],
            outputs=[],
        )

        send.click(predict,#这个是点击
                    inputs=[
                        message,  top_k, history_len, temperature,
                        top_p, state
                    ],
                    outputs=[message, chatbot, state])
        clear_history.click(fn=clear_session,
                            inputs=[],
                            outputs=[chatbot, state],
                            queue=False)

        message.submit(predict,#这个是触发回车事件
                        inputs=[
                            message, top_k, history_len,
                            temperature, top_p, state
                        ],
                        outputs=[message, chatbot, state])
    #这5个模块：load_model_button,init_vs,send,clear_history,message.submit都是触发事件的，这些事件都是在点击或者回车时触发的
    demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
